# Image Classification ML Assignment 

#### In this jupyter notebook, you will modify and run a machine learning model to classify humpback whale individuals. This script has several functions that are written for you, please do NOT modify any code unless it specifies to change it. 

#### Training the first model will take roughly 10 minutes. Although there are questions throughout this file, it is prudent to begin training your model before answering these questions. Once the model has begun training, explore the structure of the image folders, consider what each box of code is doing, and answer the questions.

In [ ]:
## Importing packages - Please DO NOT alter this box ##
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data.sampler import WeightedRandomSampler

from captum.attr import IntegratedGradients
from captum.attr import DeepLift
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz

import torchvision
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

import os
import imageio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm
from scipy.ndimage import binary_erosion
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns

#import wandb #remove comment if you are using weights and biases
#import random #remove comment if you are using weights and biases

**What is the goal of transforming the data set?**

*Put your answer here*

In [ ]:
images = "/projects/bgmp/shared/Bi625/ML_Assignment/Datasets/Humpback_ML_set/Humpback_ML_set"

In [ ]:
transform = transforms.Compose([
            transforms.Resize([224,224]), # Resizing the image as the VGG only take 224 x 244 as input size
            transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize the data, these are the values that ResNet suggests based on their training data (natural scences)
            transforms.RandomRotation(10),
            transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 1)),
    transforms.ColorJitter(brightness=0.05, contrast=0.05),
    transforms.RandomHorizontalFlip()])

The below code completes the image transformations as coded in the above box.

In [ ]:
all_images = datasets.ImageFolder(images, transform )
print(len(all_images))
print(all_images)

The below code eastablishes the data split between training, validation, and testing.

In [ ]:
train_size = int(0.7 * len(all_images))
val_size = int(0.15 * len(all_images))
test_size = len(all_images) - (train_size + val_size)
print(train_size, val_size, test_size)
assert train_size + val_size + test_size == len(all_images)

In [ ]:
train_set, val_set, test_set = torch.utils.data.random_split(all_images, [train_size, val_size, test_size])

The below function is used to get weights for the image classes.

In [ ]:
def _get_weights(subset,full_dataset):
    ys = np.array([y for _, y in subset])
    counts = np.bincount(ys)
    label_weights = 1.0 / counts
    weights = label_weights[ys]

    print("Number of images per class:")
    for c, n, w in zip(full_dataset.classes, counts, label_weights):
        print(f"\t{c}:\tn={n}\tweight={w}")
        
    return weights



The below function is used to randomly select dataset images to evaluate how the transformations altered the images.

In [ ]:
def show_random_dataset_image(dataset):
    idx = np.random.randint(0, len(dataset))    # take a random sample
    img, mask = dataset[idx]                    # get the image and the nuclei masks
    f, axarr = plt.subplots(1, 2)               # make two plots on one figure
    axarr[0].imshow(img[0], cmap="gist_ncar")                     # show the image, cmap is the color map that the image is being shown in
    #axarr[1].imshow(mask[0])                    # show the masks
    _ = [ax.axis('off') for ax in axarr]        # remove the axes
    print('Image size is %s' % {img[0].shape})
    print(img.shape)
    plt.show()

**What is the difference between image size and torch image size?**

*Put your answer here*

In [ ]:
show_random_dataset_image(train_set)
show_random_dataset_image(train_set)
show_random_dataset_image(train_set)
show_random_dataset_image(train_set)


show_random_dataset_image(val_set)
show_random_dataset_image(test_set)

**Add a new transformation to the training data and discuss how it modifies the images.**

*Put your answer here*

In [ ]:
transform = transforms.Compose([
            transforms.Resize([224,224]), # Resizing the image as the VGG only take 224 x 244 as input size
            transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize the data, these are the values that ResNet suggests based on their training data (natural scences)
            transforms.RandomRotation(10),
            transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 1)),
    transforms.ColorJitter(brightness=0.05, contrast=0.05),
    transforms.RandomHorizontalFlip()])

all_images = datasets.ImageFolder(images, transform )

train_set, val_set, test_set = torch.utils.data.random_split(all_images, [train_size, val_size, test_size])

show_random_dataset_image(train_set)
show_random_dataset_image(train_set)
show_random_dataset_image(train_set)
show_random_dataset_image(train_set)
show_random_dataset_image(val_set)
show_random_dataset_image(test_set)

In [ ]:
train_weights = _get_weights(train_set,all_images)
train_sampler = WeightedRandomSampler(train_weights, len(train_weights))


**When the class weight is calculated by WeightedRandomSampler what is it used for in the training of a model?**

<details>
  <summary>Hint</summary>
  
Check out this resource: https://towardsdatascience.com/demystifying-pytorchs-weightedrandomsampler-by-example-a68aceccb452 


*Put your answer here*

In [ ]:
train_loader = DataLoader(train_set, batch_size=12, drop_last=True, sampler=train_sampler)
val_loader = DataLoader(val_set, batch_size=12, drop_last=True, shuffle=True)
test_loader = DataLoader(test_set, batch_size=12, drop_last=True, shuffle=True)

**What is resnet18?**

<details>
  <summary> Hint </summary>
  
 Check out this website: https://www.geeksforgeeks.org/resnet18-from-scratch-using-pytorch/

*Put your answer here*

In [ ]:
resnet18_model = torchvision.models.resnet18(weights = False, progress  = True, num_classes=5)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18_model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet18_model.to(device)
print(f"Will use device {device} for training")


The below functions are used to train, evaluate, and test the Machine Learning model.

In [ ]:
def train(model,loss,train_dataloader):
    model.train()
    epoch_loss = 0
    num_batches = 0
    for x, y in tqdm(train_dataloader):

        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        y_pred = model(x)
        l = loss(y_pred, y)
        l.backward()
        optimizer.step()

        epoch_loss += l
        num_batches += 1

    return epoch_loss/num_batches

def evaluate(model, loss, dataloader):
    
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in tqdm(dataloader):

            x, y = x.to(device), y.to(device)

            logits = model(x)
            val_loss = loss(logits,y)
            
            probs = torch.nn.Softmax(dim=1)(logits)
            predictions = torch.argmax(probs, dim=1)

            correct += int(torch.sum(predictions == y).cpu().detach().numpy())
            total += len(y)

        accuracy = correct/total

    return accuracy, val_loss

def validate(model,loss, validation_dataloader):
    '''Evaluate prediction accuracy on the validation dataset.'''
    
    model.eval()
    return evaluate(model,loss,validation_dataloader)

def test(model,loss,test_dataloader):
    '''Evaluate prediction accuracy on the test dataset.'''
    
    model.eval() 
    return evaluate(model, loss,test_dataloader)


*Put your answer here*

#### Start with 10 epochs, run the below model and consider the success of the model training. 

**What is your starting accuracy and final accuracy? Why is there a difference between the two?**

In [ ]:
# wandb.init(
#     project="Humpback-classification-project",
#     name="Humpback-training-run-preset-hyperparameters",
#     config={"learning rate":0.001,
#         "architecture": "CNN",
#         "dataset": "Humpback",
#         "epochs":10}
# ) ### Remove comments if using weights and biases

###
step = 0
epochs = 10

validation_accuracy_list = []
validation_loss_list = []

for epoch in range(epochs+1):
    while step < epoch: 
    
        epoch_loss = train(resnet18_model,loss_fn,train_loader)
        print(f"epoch {epoch}, training loss={epoch_loss}")
    
        validation_accuracy, validation_loss = validate(resnet18_model, loss_fn,val_loader)
        validation_accuracy_list.append(validation_accuracy)
        validation_loss_list.append(validation_loss)
        print(f"epoch {epoch}, validation accuracy={validation_accuracy}")
        
        # log metrics to wandb - remove comments if using weights and biases
        #wandb.log({"validation_accuracy": validation_accuracy, "validation_loss": validation_loss, "epoch_loss":epoch_loss})

        step += 1

plt.plot(range(10), validation_accuracy_list, color = "magenta")
plt.xlabel("Epoch number")
plt.ylabel("Valiation accuracy")
plt.show()  

plt.plot(range(10), validation_loss_list, color = "purple")
plt.xlabel("Epoch number")
plt.ylabel("Valiation loss")
plt.show()  

#wandb.finish() #remove comments if using weights and biases


After running the model and calculating the overall accuracy, we can examine the data via a confusion matrix which highlights the accuracy by class.

In [ ]:
# predict the test dataset
def predict(model, dataset):
    dataset_prediction = []
    dataset_groundtruth = []
    model = model
    with torch.no_grad():
        for x, y_true in dataset:
            inp = x[None]
            y_pred = model(inp)
            dataset_prediction.append(y_pred.argmax().cpu().numpy())
            dataset_groundtruth.append(y_true)
    
    return np.array(dataset_prediction), np.array(dataset_groundtruth)
            
    # create seaborn heatmap with required labels
    ax=sns.heatmap(cm, annot=annot, fmt='', vmax=30, xticklabels=x_axis_labels, yticklabels=y_axis_labels)
    ax.set_title(title)

# Plot confusion matrix 
# orginally from Runqi Yang; 
# see https://gist.github.com/hitvoice/36cf44689065ca9b927431546381a3f7
def cm_analysis(y_true, y_pred, title, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    labels = ['new_whale','w_0369a5c', 'w_23a388d', 'w_9b5109b', 'w_9c506f6']
    cm = confusion_matrix(y_true, y_pred)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
                              
    x_axis_labels = ['new_whale','w_0369a5c', 'w_23a388d', 'w_9b5109b', 'w_9c506f6'] # labels for x-axis
    y_axis_labels = ['new_whale','w_0369a5c', 'w_23a388d', 'w_9b5109b', 'w_9c506f6'] # labels for y-axis
    ax=sns.heatmap(cm, annot=annot, fmt='', vmax=30, xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap = "viridis")
    ax.set_title(title)

## This plot only contains the test set of data
## The test set of data has not been seen by the model yet
y_pred, y_true = predict(resnet18_model, test_set)
cm_analysis(y_true, y_pred, "Confusion matrix")

Since the dataset is small, we can also manually look at the true and predicted classifications for our test dataset. 

In [ ]:
test_pred, test_true = predict(resnet18_model, test_set)
print(test_pred)
print(test_true)

### What features did the model use to make decisions?

Based on the confusion matrices and validation loss, it is clear that the model is learning. To determine the features that are involved in decision making, there are a few possible approaches. Two options are shown below, integrated gradients and CycleGAN. 

#### Understanding feature importance using Integrated Gradients

Integrated gradients is an attribution method. Attribution methods score the input data based on the prediction that the model makes using scores for each feature. The gradient is the signal that tells the network how much to increase or decrease a certain weight in the network during backpropogation. These gradients are overlaid onto the images to showcase the regions on the images that influence the weights.

**Run the below code. Does the integrated gradient show what types of features the model used to classify the images? What have you learned from it?**

*Put your answer here*

In [ ]:
def visualize_ig(idx, 
                 _train_dataset=train_set, 
                 _test_dataset=test_set):
        
    # Get corresponding input and target tensors:
    input_tensor = _test_dataset[idx][0].unsqueeze(0)
    input_tensor.requires_grad = True
    target = _test_dataset[idx][1]
    
    # We will use the IntegratedGradients algorithm:
    algorithm = IntegratedGradients(resnet18_model)
    
    # First we clear the gradients from the model:
    resnet18_model.zero_grad()
    
    # Run attribution:
    attr_ig, delta = algorithm.attribute(input_tensor,
                                          target=target,
                                          baselines=input_tensor * 0,
                                          return_convergence_delta=True
                                        )
    
    # Integrated Gradients:
    attr_ig = np.transpose(attr_ig[0].cpu().detach().numpy(), (1, 2, 0))
    
    # Original image:
    original_image = np.transpose((_test_dataset[idx][0].detach().numpy() * 0.5) + 0.5, (1, 2, 0))
    
    print("Annotated Whale Individual", test_true[idx])
    plt.imshow(original_image)
    
    
    print("Predicted Whale Individual", test_pred[idx])


    # This visualises the attribution of labels to 
    viz.visualize_image_attr(attr_ig, 
                             original_image, 
                             method="blended_heat_map",
                             sign="all",
                             show_colorbar=True, 
                             title="Overlayed Integrated Gradients")

visualize_ig(0)

visualize_ig(8)

visualize_ig(20)

visualize_ig(32)

## Code and Explore! 

Now that you have a sense for how to train a machine learning model, your task is to improve the success of our classification model.

**Make one (or more) modifications to the model training process, re-train the model, and report out your changes and whether they increased model accuracy. Do you have ideas why your modifications were/were not successful?**

Types of changes that you can make include changing the model, the number of epochs, the optimizer, loss function, learning rate, batch size, image transformations, and more. If you decide to change any lines of code that are not in the below box, please add them to the box (DO NOT MODIFY ANY CODE ABOVE THIS MARKDOWN BOX). 

If your changes are ambitious, the code may take hours to run. If this is the case, please consider turning this notebook into a python script and running it via sBATCH. 

**Please note, you will NOT be graded by the success of your classifier! It is okay if you are unable to improve the accuracy of the classifier**

*Put your answer here*

In [ ]:
train_loader = DataLoader(train_set, batch_size=12, drop_last=True, sampler=train_sampler)
val_loader = DataLoader(val_set, batch_size=12, drop_last=True, shuffle=True)
test_loader = DataLoader(test_set, batch_size=12, drop_last=True, shuffle=True)

resnet18_model = torchvision.models.resnet18(weights = False, progress  = True, num_classes=5)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18_model.parameters(), lr=0.001) #possibly update

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet18_model.to(device)
print(f"Will use device {device} for training")

# wandb.init(
#     project="Humpback-classification-project",
#     name="Humpback-training-run-student-selected-hyperparameters", ##update this with your first initial and last name
#     config={"learning rate":.001, # possibly update
#         "architecture": "CNN",
#         "dataset": "Humpback",
#         "epochs": 10}  # possibly update
# ) ##Remove comments if using weights and biases



step = 0
epochs = 10 # possibly update

validation_accuracy_list = []
validation_loss_list = []

for epoch in range(epochs+1):
    while step < epoch: 
    
        epoch_loss = train(resnet18_model,loss_fn,train_loader)
        print(f"epoch {epoch}, training loss={epoch_loss}")
    
        validation_accuracy, validation_loss = validate(resnet18_model, loss_fn,val_loader)
        validation_accuracy_list.append(validation_accuracy)
        validation_loss_list.append(validation_loss)
        print(f"epoch {epoch}, validation accuracy={validation_accuracy}")
    
        # log metrics to wandb - remove comments if using weights and biases
        #wandb.log({"validation_accuracy": validation_accuracy, "validation_loss": validation_loss, "epoch_loss":epoch_loss})

        step += 1


#wandb.finish() #remove comments if using weights and biases

plt.plot(range(10), validation_accuracy_list, color = "magenta")
plt.xlabel("Epoch number")
plt.ylabel("Valiation accuracy")
plt.show()  

plt.plot(range(10), validation_loss_list, color = "purple")
plt.xlabel("Epoch number")
plt.ylabel("Valiation loss")
plt.show()  

y_pred, y_true = predict(resnet18_model, test_set)
cm_analysis(y_true, y_pred, "Confusion matrix")

## Integrated gradient 
visualize_ig(0)
visualize_ig(8)
visualize_ig(20)
visualize_ig(32)